In [94]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords

# import langid
# from langdetect import detect
# from langdetect import DetectorFactory



temp = ""
pattern1 = '__label__1'
pattern2 = '__label__2'
star = list()
review = list()
title = list()

with open("./test.ft.txt", "r") as file:
    for line in file:
        if re.findall(pattern1,line) == [] and re.findall(pattern2,line) == []:
            temp= temp + ' ' + line
        elif re.findall(pattern1,line) != []:
            if temp!='':review.append(temp)
            temp = re.findall('.*: (.*)',line)[0]
            title.append(re.findall('__label__1 (.*):',line)[0])
            star.append(pattern1)
        else:
            if temp!='':review.append(temp)
            temp = re.findall('.*: (.*)',line)[0]
            title.append(re.findall('__label__2 (.*):',line)[0])
            star.append(pattern2)
    if temp!='': review.append(temp)

amazon = pd.DataFrame({"Title":title,"Ratings":star,"Review":review})
amazon['ratings_numeral'] = amazon['Ratings'].map({"__label__1":0,"__label__2":1})
amazon['Review'] = amazon['Review'].astype(str)
# amazon = amazon[amazon['Review'].str.strip() != ""]
# amazon['language'] = amazon['Review'].apply(detect)
# amazon.head()

In [95]:
import fasttext


model = fasttext.load_model('lid.176.bin')

In [96]:
def fasttext_detect(text):
    """FastText-based language detection."""
    return model.predict(text)[0][0].replace("__label__", "")


try:
    amazon['language'] = amazon['Review'].apply(fasttext_detect)
except:
    amazon['language'] = 'unknown'

In [64]:
unsupported_languages = ['ceb', 'si', 'als', 'fy', 'sh', 'nds', 'mk','ja','oc','cy','hr','ceb','la','pl','vi','cs','lt','fa','uk','hy','ms','eo']
amazon = amazon[~amazon['language'].isin(unsupported_languages)]


In [65]:
amazon.head()

,Title,Ratings,Review,ratings_numeral,language
0,Great CD,__label__2,My lovely Pat has one of the GREAT voices of h...,4.5,en
1,One of the best game music soundtracks - for a...,__label__2,Despite the fact that I have only played a sma...,4.5,en
2,Batteries died within a year ...,__label__1,I bought this charger in Jul 2003 and it worke...,1.5,en
3,"works fine, but Maha Energy is better",__label__2,Check out Maha Energy's website. Their Powerex...,4.5,en
4,Great for the non-audiophile,__label__2,Reviewed quite a bit of the combo players and ...,4.5,en


In [66]:
amazon['language'].unique()

array(['en', 'es', 'fr', 'pt', 'it', 'ca', 'zh', 'sv', 'ru', 'de', 'nl',
       'hu', 'id', 'ro', 'ko', 'sl', 'da', 'tr', 'eu', 'fi', 'no', 'el',
       'ar'], dtype=object)

In [69]:
print(stopwords.fileids())

['albanian', 'arabic', 'azerbaijani', 'basque', 'belarusian', 'bengali', 'catalan', 'chinese', 'danish', 'dutch', 'english', 'finnish', 'french', 'german', 'greek', 'hebrew', 'hinglish', 'hungarian', 'indonesian', 'italian', 'kazakh', 'nepali', 'norwegian', 'portuguese', 'romanian', 'russian', 'slovene', 'spanish', 'swedish', 'tajik', 'turkish']


In [97]:
amazon = amazon[amazon['language']=='en']
amazon['language'].unique()
ignore_words = stopwords.words('english')

amazon.head()





,Title,Ratings,Review,ratings_numeral,language
0,Great CD,__label__2,My lovely Pat has one of the GREAT voices of h...,1,en
1,One of the best game music soundtracks - for a...,__label__2,Despite the fact that I have only played a sma...,1,en
2,Batteries died within a year ...,__label__1,I bought this charger in Jul 2003 and it worke...,0,en
3,"works fine, but Maha Energy is better",__label__2,Check out Maha Energy's website. Their Powerex...,1,en
4,Great for the non-audiophile,__label__2,Reviewed quite a bit of the combo players and ...,1,en


In [98]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


amazon.info()

<class 'pandas.core.frame.DataFrame'>
Index: 398427 entries, 0 to 399999
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Title            398427 non-null  object
 1   Ratings          398427 non-null  object
 2   Review           398427 non-null  object
 3   ratings_numeral  398427 non-null  int64 
 4   language         398427 non-null  object
dtypes: int64(1), object(4)
memory usage: 18.2+ MB


In [99]:
def pre_process(text):
    text = text.lower()
    text = re.sub(r'<.*?>','',text)
    text = re.sub(r'[^a-z\s]','',text)
    words = text.split()
    words = [word for word in words if word not in ignore_words]
    return " ".join(words)

    
amazon['clean_review'] = amazon['Review'].apply(pre_process)

In [100]:
amazon.head()

,Title,Ratings,Review,ratings_numeral,language,clean_review
0,Great CD,__label__2,My lovely Pat has one of the GREAT voices of h...,1,en,lovely pat one great voices generation listene...
1,One of the best game music soundtracks - for a...,__label__2,Despite the fact that I have only played a sma...,1,en,despite fact played small portion game music h...
2,Batteries died within a year ...,__label__1,I bought this charger in Jul 2003 and it worke...,0,en,bought charger jul worked ok design nice conve...
3,"works fine, but Maha Energy is better",__label__2,Check out Maha Energy's website. Their Powerex...,1,en,check maha energys website powerex mhcf charge...
4,Great for the non-audiophile,__label__2,Reviewed quite a bit of the combo players and ...,1,en,reviewed quite bit combo players hesitant due ...


In [101]:
vectorize = TfidfVectorizer(max_features=5000)
X = vectorize.fit_transform(amazon['clean_review'])


In [102]:
y = amazon['ratings_numeral']
from sklearn.linear_model import LogisticRegression


train_X,test_X, train_y,test_y = train_test_split(X,y,test_size=0.2,random_state=3)
model = LogisticRegression(random_state=4)
model.fit(train_X,train_y)





LogisticRegression(random_state=4)

In [103]:
prediction = model.predict(test_X)
print(f'Model Accuracy: {accuracy_score(prediction,test_y)}')

Model Accuracy: 0.8577165374093316


In [104]:
amazon['clean_title'] = amazon['Title'].apply(pre_process)
amazon['combined'] = amazon['clean_review'] + amazon['clean_title']
X_new = vectorize.fit_transform(amazon['combined'])

In [114]:
train_X_new,test_X_new,train_y_new,test_y_new = train_test_split(X_new,y,test_size=0.2,random_state=5)
model.fit(train_X_new,train_y_new)


LogisticRegression(random_state=4)

In [115]:
predictions_new = model.predict(test_X_new)
print(f"Accuracy score : {accuracy_score(predictions_new,test_y_new)}")


Accuracy score : 0.8643048967196245
